In [ ]:
from z3 import *

# ============================ 共用工具 ============================

def z3_to_float(z3_val):
    """Z3 數值轉 Python float，若為無限小數用 as_decimal() 截取。"""
    try:
        dec_str = z3_val.as_decimal(20)
        if dec_str.endswith('?'):
            dec_str = dec_str[:-1]
        return float(dec_str)
    except Exception:
        return float(str(z3_val))

# ------------------ 證券交易稅 ------------------

def calculate_securities_tax_item(
    tax_item: str,
    tp: float,
    ep: float | None = None,
    sc: int | None = None,
    *,
    free_vars: list | None = None,
):
    """計算/最佳化證券交易稅。

    free_vars 允許 "tp", "ep", "sc" 任意組合，代表可調整參數。
    回傳字串固定保留 2 位 (截斷)。
    """
    if free_vars is None:
        free_vars = []
    free_vars = set(free_vars)

    # 建立 Optimizer
    opt = Optimize()

    tp_z = Real("tp")
    opt.add(tp_z >= 0)
    if "tp" not in free_vars:
        opt.add(tp_z == tp)

    ep_z, sc_z = None, None
    if tax_item in {"warrant_delivery_stock", "warrant_delivery_cash"}:
        if ep is None or sc is None:
            raise ValueError("計算交割/結算須提供 ep 與 sc")
        ep_z = Real("ep")
        sc_z = Real("sc")
        opt.add(ep_z >= 0, sc_z >= 0)
        if "ep" not in free_vars:
            opt.add(ep_z == ep)
        if "sc" not in free_vars:
            opt.add(sc_z == sc)

    # 稅式
    if tax_item == "stock":
        tax_expr = tp_z * 0.003
    elif tax_item == "bond":
        tax_expr = tp_z * 0.001
    elif tax_item == "warrant":
        tax_expr = tp_z * 0.001
    elif tax_item == "warrant_delivery_stock":
        tax_expr = ep_z * sc_z * 0.003
    elif tax_item == "warrant_delivery_cash":
        tax_expr = ep_z * sc_z * 0.001
    else:
        raise ValueError("未知的證券交易稅項目")

    tax_int = Int("tax_int")
    opt.add(tax_int == ToInt(tax_expr), tax_int >= 0)
    opt.minimize(tax_int)

    if opt.check() != sat:
        return None

    m = opt.model()
    best_tax = m[tax_int].as_long()
    truncated = best_tax / 100  # 已經是整數 (元) -> /100 得到 2 位
    return f"{truncated:.2f}"

# ------------------ 期貨交易稅 ------------------

def calculate_futures_tax_item(
    tax_item: str,
    ca: float,
    pa: float | None = None,
    *,
    free_vars: list | None = None,
):
    """計算/最佳化期貨交易稅。free_vars 可包含 "ca", "pa"。"""
    if free_vars is None:
        free_vars = []
    free_vars = set(free_vars)

    opt = Optimize()

    ca_z = Real("ca")
    opt.add(ca_z >= 0)
    if "ca" not in free_vars:
        opt.add(ca_z == ca)

    pa_z = None
    if tax_item == "option":
        if pa is None:
            raise ValueError("option 必須給 pa")
        pa_z = Real("pa")
        opt.add(pa_z >= 0)
        if "pa" not in free_vars:
            opt.add(pa_z == pa)

    # 稅式
    if tax_item == "stock_index":
        tax_expr = ca_z * 0.00002
    elif tax_item == "interest_rate_30":
        tax_expr = ca_z * 0.000000125
    elif tax_item == "interest_rate_10":
        tax_expr = ca_z * 0.00000125
    elif tax_item == "option":
        tax_expr = pa_z * 0.001
    elif tax_item == "gold":
        tax_expr = ca_z * 0.0000025
    else:
        raise ValueError("未知的期貨交易稅項目")

    tax_int = Int("tax_int")
    opt.add(tax_int == ToInt(tax_expr), tax_int >= 0)
    opt.minimize(tax_int)

    if opt.check() != sat:
        return None

    best_tax = opt.model()[tax_int].as_long()
    truncated = best_tax / 100
    return f"{truncated:.2f}"

# ------------------ Demo ------------------
if __name__ == "__main__":
    print("=== 證券交易稅範例 ===")
    r1 = calculate_securities_tax_item("stock", 100_000)
    print("固定參數:", r1)
    r2 = calculate_securities_tax_item("stock", 100_000, free_vars=["tp"])
    print("tp 為自由變數 -> 最小稅額:", r2)

    print("\n=== 期貨交易稅範例 ===")
    r3 = calculate_futures_tax_item("option", 5_000_000, pa=20_000)
    print("固定參數:", r3)
    r4 = calculate_futures_tax_item("option", 5_000_000, pa=20_000, free_vars=["pa"])
    print("pa 為自由變數 -> 最小稅額:", r4)
